In [1]:
# Configure Spark Session

from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName('OlistData')\
.getOrCreate()

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
0,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Set path to the S3 bucket where data is stored
s3_path = "s3://kalan-emr-tutorial/input/hadoop/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Create DataFrames for each CSV file
customers_df = spark.read.csv(s3_path + "olist_customers_dataset.csv", header=True, inferSchema=True)
geolocation_df = spark.read.csv(s3_path + "olist_geolocation_dataset.csv", header=True, inferSchema=True)
order_items_df = spark.read.csv(s3_path + "olist_order_items_dataset.csv", header=True, inferSchema=True)
payments_df = spark.read.csv(s3_path + "olist_order_payments_dataset.csv", header=True, inferSchema=True)
order_reviews_df = spark.read.csv(s3_path + "olist_order_reviews_dataset.csv", header=True, inferSchema=True)
orders_df = spark.read.csv(s3_path + "olist_orders_dataset.csv", header=True, inferSchema=True)
products_df = spark.read.csv(s3_path + "olist_products_dataset.csv", header=True, inferSchema=True)
sellers_df = spark.read.csv(s3_path + "olist_sellers_dataset.csv", header=True, inferSchema=True)
category_translation_df = spark.read.csv(s3_path + "product_category_name_translation.csv", header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Cache Frequently Used Data
customers_df.cache()
order_items_df.cache()
orders_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[order_id: string, customer_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp]

In [21]:
order_items_joined_df = orders_df.join(order_items_df,'order_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
orders_items_products_df = order_items_joined_df.join(products_df,'product_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
orders_items_products_sellers_df = orders_items_products_df.join(sellers_df, 'seller_id', 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Geolocation Data

full_orders_df = full_orders_df.join(geolocation_df,full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
full_orders_df = full_orders_df.join(order_reviews_df,'order_id', 'left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
full_orders_df = full_orders_df.join(payments_df, 'order_id', 'left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Total Revenues Per Seller
from pyspark.sql.functions import *
seller_revenue_df = full_orders_df.groupBy('seller_id').agg(sum('price').alias('total_revenue'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
seller_revenue_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+
|           seller_id|      total_revenue|
+--------------------+-------------------+
|8e6cc767478edae94...|  1145757.400000003|
|b76dba6c951ab00dc...|  302582.6600000009|
|33cbbec1e7e1044aa...| 190508.31999999887|
|7a67c85e85bb2ce85...|2.031279488999876E7|
|3d8fa2f5b647373c8...|  458520.1200000021|
+--------------------+-------------------+
only showing top 5 rows

In [40]:
# Total Orders Per customer

customer_orders_df = full_orders_df.groupBy('customer_id')\
    .agg(count('order_id').alias('total_orders'))\
    .orderBy(desc('total_orders'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
customer_orders_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
|d3e82ccec3cb5f956...|        6876|
|d5f2b3f597c7ccafb...|        6706|
|c2f18647725395af4...|        6612|
|24e7dc2ff8c071263...|        6597|
|7bb57d182bdc11653...|        6258|
|63b964e79dee32a35...|        6072|
|d22f25a9fadfb1abb...|        6072|
|1ff773612ab8934db...|        5820|
|13aa59158da63ba0e...|        5206|
|78fc46047c4a639e8...|        5200|
|dd3f1762eb601f41c...|        4992|
|a193aa8d905b8e246...|        4896|
|9eb3d566e87289dcb...|        4872|
|2ba91e12e5e4c9f56...|        4752|
|55e7cfd6e28d2fbfb...|        4728|
+--------------------+------------+
only showing top 20 rows

In [42]:
# Average Review Score Per Seller
seller_reviews_df = full_orders_df.groupBy('seller_id')\
    .agg(avg('review_score').alias('average_review_score'))\
    .orderBy(desc('average_review_score'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
seller_reviews_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|           seller_id|average_review_score|
+--------------------+--------------------+
|73da1880b39c71de2...|                 5.0|
|48efc9d94a9834137...|                 5.0|
|9d213f303afae4983...|                 5.0|
|a20d8058c866dbaec...|                 5.0|
|c3fe93ba3085f9285...|                 5.0|
|57a834594e1e9e921...|                 5.0|
|63272377184e96994...|                 5.0|
|4a82c4af97ffc0fb2...|                 5.0|
|d263fa444c1504a75...|                 5.0|
|7238f877570096ae4...|                 5.0|
|2bd05d410a8fd26dc...|                 5.0|
|a81466620e46a6970...|                 5.0|
|45a3d05fb00435e52...|                 5.0|
|8c351ed7c326c6212...|                 5.0|
|2d34636518ba88f53...|                 5.0|
|228e4c1a0be164f61...|                 5.0|
|2063dd1b9f568df1c...|                 5.0|
|8c9348f33ae3dada2...|                 5.0|
|3f84fefc07dc159b2...|                 5.0|
|e439f7176d763f92d...|          

In [44]:
# Most Sold Products (top 10)
top_products_df = full_orders_df.groupBy('product_id', 'product_category_name')\
    .agg(count('order_id').alias('total_sales'))\
    .orderBy(desc('total_sales'))\
    .limit(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
top_products_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+-----------+
|          product_id|product_category_name|total_sales|
+--------------------+---------------------+-----------+
|aca2eb7d00ea1a7b8...|     moveis_decoracao|      86740|
|422879e10f4668299...|   ferramentas_jardim|      81110|
|99a4788cb24856965...|      cama_mesa_banho|      78775|
|389d119b48cf3043d...|   ferramentas_jardim|      60248|
|d1c427060a0f73f6b...| informatica_acess...|      59274|
|368c6c730842d7801...|   ferramentas_jardim|      58358|
|53759a2ecddad2bb8...|   ferramentas_jardim|      52654|
|53b36df67ebb7c415...|   relogios_presentes|      52105|
|154e7e31ebfa09220...|         beleza_saude|      42700|
|3dd2a17168ec895c7...| informatica_acess...|      40787|
+--------------------+---------------------+-----------+

In [46]:
# Top Spending Customers
top_customers_df = full_orders_df.groupBy('customer_id', 'customer_city', 'customer_state')\
    .agg(sum('price').alias('total_spent'))\
    .orderBy(desc('total_spent'))\
    .limit(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
top_customers_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------+------------------+
|         customer_id| customer_city|customer_state|       total_spent|
+--------------------+--------------+--------------+------------------+
|d3e82ccec3cb5f956...|       niteroi|            RJ|         6662844.0|
|df55c14d1476a9a34...|      araruama|            RJ|         3565657.0|
|fe5113a38e3575c04...|       niteroi|            RJ|         3293604.0|
|ec5b2ba62e5743423...|    vila velha|            ES|         2556120.0|
|63b964e79dee32a35...|       atibaia|            SP|         2501664.0|
|46bb3c0b1a65c8399...|   santa maria|            RS|         2336752.0|
|05455dfa7cd02f13d...|   divinopolis|            MG| 2160194.400000087|
|3690e975641f01bd0...|  porto seguro|            BA|         2124498.0|
|349509b216bd5ec11...|      araruama|            RJ|         1923627.0|
|695476b5848d64ba0...|rio de janeiro|            RJ|1820543.1299999943|
+--------------------+--------------+--------------+------------

# Optimized Joins for Data Integration

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…